<a href="https://colab.research.google.com/github/purvadanait/Final-Year-Project/blob/main/inference_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!ls "/content/drive/My Drive/cv2_utils.py"

'/content/drive/My Drive/cv2_utils.py'


In [11]:
!cat "/content/drive/My Drive/cv2_utils.py"

# -*- coding: utf-8 -*-
"""cv2_utils.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1nM7TkHCvT0O8AzmfaLJFHpQhRuQSTfGD
"""

from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
from functools import reduce
from scipy.interpolate import interp1d
import math

def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def extract_bboxes(fused):
    """Compute bounding boxes from masks.
    mask: [height, width]..
    Returns: bbox array [num_instances, (y1, x1, y2, x2)].
    """
    mask = cv2.cvtColor(fused, cv2.COLOR_BGR2GRAY)
    mask[mask < 40] = 0
    mask[mask >= 40] = 1
    mask = mask.reshape(256, 256, 1)
    boxes = np.zeros([mask.shape[-1], 4], dtype=np.int32)
    for i in range(mask.shape[-1]):
        m = mask[:, :, i]
        # Bounding box.
        horizontal_indici

In [15]:
import sys
sys.path.append("/content/drive/My Drive")

In [16]:
import cv2_utils

In [18]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from io import BytesIO
from cv2_utils import getContours
import torchvision.transforms as transforms
from .models.deepcrack_model import DeepCrackModel

ImportError: ignored